# ✔️ Apply Topic Modeling algorithms using Gensim — before moving on to more advanced textual analysis techniques

> Topic Modelling is a great way to analyze completely unstructured textual data. Python NLP framework such as Gensim, NLTK, and spaCy makes it easier to do this.

> The purpose of this article is to guide one through the whole process of topic modeling right from pre-processing your raw textual data, creating your topic models, evaluating the topic models, to visualizing them. The python packages used during the tutorial will be spaCy (for pre-processing), Gensim (for topic modeling), and pyLDAvis (for visualization).

# 📌 Notebook Goals
> - Learn how to use the power of `spaCy` to clean textual data.
> - Use different Topic Modelling techniques like `LDA (Latent Dirichlet Allocation)`, `LSI (Latent Semantic Indexing)`, and `HDP (Hierarchical Drichlet Process)`
---

# 📚 Topic Modelling Overview
Let's understand the general concept of Topic Modelling and why it's important! 
> - Topic Modeling is an unsupervised machine learning technique that allows us to efficiently analyze large volumes of text by clustering documents into topics.
> - A large amount of text data is unlabeled meaning we can’t apply Supervised Learning approaches to create machine learning models for the data! In this case of text data, this means attempting to discover clusters of documents, grouped by topic. A very important idea to keep in mind here is that we don’t know the correct topic or the right answer! All we know is that the documents clustered together share similar topic ideas. It is up to us to identify what these topics represent.

---
# 📑 Text Analysis Tutorial

> Our steps, naturally, are setting up our imports. We will be using spaCy for data pre-processing and computational linguistics, Gensim for Topic Modeling, Scikit-Learn for classification, and Keras for text generation.

# ✔️ Import Libraries

In [ ]:
!pip install gensim=="3.8.3"
!pip install spacy=="2.3.5"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-linux_x86_64.whl size=26527989 sha256=878be611757753ee3fbdd0837e8eb46a35d5a15a4e79323c3d8de8d7b6859c70
  Stored in directory: /root/.cache/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.5 MB/s eta

In [ ]:
import os
import numpy as np

import spacy 
from spacy import displacy

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet

import matplotlib.pyplot as plt
import sklearn
import keras

import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
print(spacy.__version__)
print(gensim.__version__)

2.3.5
3.8.3


---
# 📂 Gathering Data

> The dataset we will be working with will be the Lee corpus which is a shortened of the Lee Background Corpus, and the 20NG dataset. 

In [ ]:
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
print(test_data_dir)

lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
print(lee_train_file)

text = open(lee_train_file).read()

/usr/local/lib/python3.9/dist-packages/gensim/test/test_data
/usr/local/lib/python3.9/dist-packages/gensim/test/test_data/lee_background.cor


---
# 🧹 Cleaning Data

> We can't have state-of-the-art results without data which is as good. Let's spend this section working on cleaning and understanding our data set. We will be checking out `spaCy`, an industry grade text-processing package.

This code block downloads the English language model for spaCy using the spacy.cli command-line interface.






In [ ]:
 import spacy.cli
 spacy.cli.download("en")

⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.9/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.9/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


This code block loads the pre-trained English language model from spaCy into the variable 'nlp'.






In [ ]:
nlp = spacy.load('en')

> For safe measure, let's add some stopwords. It's a newspaper corpus, so it is likely we will be coming across variations of 'said', 'Mister', and 'Mr'... which will not really add any value to the topic models.

This code block shows the creation of a custom list of stopwords and the addition of these stopwords to the spaCy default stopword list.






In [ ]:
my_stop_words = ['say', '\s', 'mr', 'Mr', 'said', 'says', 'saying', 'today', 'be']

for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

This code block initializes the NLP pipeline with the loaded English model from spaCy and processes a given text to create a Doc object. The Doc object is a container for accessing linguistic annotations and other information about the text.






In [ ]:
doc = nlp(text)

In [ ]:
doc

Hundreds of people have been forced to vacate their homes in the Southern Highlands of New South Wales as strong winds today pushed a huge bushfire towards the town of Hill Top. A new blaze near Goulburn, south-west of Sydney, has forced the closure of the Hume Highway. At about 4:00pm AEDT, a marked deterioration in the weather as a storm cell moved east across the Blue Mountains forced authorities to make a decision to evacuate people from homes in outlying streets at Hill Top in the New South Wales southern highlands. An estimated 500 residents have left their homes for nearby Mittagong. The New South Wales Rural Fire Service says the weather conditions which caused the fire to burn in a finger formation have now eased and about 60 fire units in and around Hill Top are optimistic of defending all properties. As more than 100 blazes burn on New Year's Eve in New South Wales, fire crews have been called to new fire at Gunning, south of Goulburn. While few details are available at this

# 💹 Computational Linguistics

Now that we have our doc object. We can see that the doc object now contains the entire corpus. This is important because we will be using the doc object to create our corpus for the machine learning algorithms. When creating a corpus for `gensim/scikit-learn`, we sometimes forget the incredible power which `spaCy` packs in its pipeline, so we will briefly demonstrate the same in this section with a smaller example sentence.


This code block uses spaCy to tokenize and parse the input text "Last Thursday, Manchester United defeated AC Milan at San Siro." into a structured format.






In [ ]:
sent = nlp('Last Thursday, Manchester United defeated AC Milan at San Siro.')
sent

Last Thursday, Manchester United defeated AC Milan at San Siro.

## 🔖 POS-Tagging

The **Part Of Speech (POS)** explains how a word is used in a sentence. There are eight main parts of speech — nouns, pronouns, adjectives, verbs, adverbs, prepositions, conjunctions, and interjections.

In [ ]:
for token in sent:
    print(token.text, token.pos_, token.tag_)

Last ADJ JJ
Thursday PROPN NNP
, PUNCT ,
Manchester PROPN NNP
United PROPN NNP
defeated VERB VBD
AC PROPN NNP
Milan PROPN NNP
at ADP IN
San PROPN NNP
Siro PROPN NNP
. PUNCT .


## 🔖 NER-Tagging  — (Named Entity Recognition)

**Named-entity recognition (NER)** is a subtask of information extraction that seeks to locate and classify named entities mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.

In [ ]:
for token in sent:
    print(token.text, token.ent_type_)

Last DATE
Thursday DATE
, 
Manchester ORG
United ORG
defeated 
AC ORG
Milan ORG
at 
San GPE
Siro GPE
. 


In [ ]:
for ent in sent.ents:
    print(ent.text, ent.label_)

Last Thursday DATE
Manchester United ORG
AC Milan ORG
San Siro GPE


In [ ]:
displacy.render(sent, style='ent', jupyter=True)

## 🧮 Dependency Parsing

The term Dependency Parsing (DP) refers to the process of examining the dependencies between the phrases of a sentence in order to determine its grammatical structure.

In [ ]:
for chunk in sent.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

Manchester United United nsubj defeated
AC Milan Milan dobj defeated
San Siro Siro pobj at


In [ ]:
for token in sent:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
         [child for child in token.children])

Last amod Thursday PROPN []
Thursday npadvmod defeated VERB [Last]
, punct defeated VERB []
Manchester compound United PROPN []
United nsubj defeated VERB [Manchester]
defeated ROOT defeated VERB [Thursday, ,, United, Milan, at, .]
AC compound Milan PROPN []
Milan dobj defeated VERB [AC]
at prep defeated VERB [Siro]
San compound Siro PROPN []
Siro pobj at ADP [San]
. punct defeated VERB []


This code block visualizes the dependency parse tree of a given sentence using displacy module in Jupyter notebook. The style is set to 'dep' and the options parameter is used to adjust the distance between words.






In [ ]:
displacy.render(sent, style='dep', jupyter=True, options={'distance':90})

# 🧹 Continuing Cleaning

> Have a quick look at the output of the doc object. It seems like nothing, right? But spaCy's internal data structure has done all the work for us. Let's see how we can create our corpus.

This code block performs preprocessing on a given document by removing stopwords, punctuation, newline characters and numbers. The processed text is then split into individual words and added to a list of lists called 'texts'.






In [ ]:
# We add some words to the stop word list
texts, article = [], []

for word in doc:
    
    if word.text != '\n' and not word.is_stop and not word.is_punct and not word.like_num and word.text != 'I':
        article.append(word.lemma_)
        
    if word.text == '\n':
        texts.append(article)
        article = []
        
        
print(texts[0])

['hundred', 'people', 'force', 'vacate', 'home', 'Southern', 'Highlands', 'New', 'South', 'Wales', 'strong', 'wind', 'push', 'huge', 'bushfire', 'town', 'Hill', 'new', 'blaze', 'near', 'Goulburn', 'south', 'west', 'Sydney', 'force', 'closure', 'Hume', 'Highway', '4:00pm', 'AEDT', 'marked', 'deterioration', 'weather', 'storm', 'cell', 'move', 'east', 'Blue', 'Mountains', 'force', 'authority', 'decision', 'evacuate', 'people', 'home', 'outlying', 'street', 'Hill', 'New', 'South', 'Wales', 'southern', 'highland', 'estimated', 'resident', 'leave', 'home', 'nearby', 'Mittagong', 'New', 'South', 'Wales', 'Rural', 'Fire', 'Service', 'weather', 'condition', 'cause', 'fire', 'burn', 'finger', 'formation', 'ease', 'fire', 'unit', 'Hill', 'optimistic', 'defend', 'property', 'blaze', 'burn', 'New', 'Year', 'Eve', 'New', 'South', 'Wales', 'fire', 'crew', 'call', 'new', 'fire', 'Gunning', 'south', 'Goulburn', 'detail', 'available', 'stage', 'fire', 'authority', 'close', 'Hume', 'Highway', 'direction

> - And this is the magic of spaCy - just like that, we've managed to get rid of stopwords, puctuation markers, and added lemmatized word.
> - Sometimes topic modeling make more sense when `New` and `York` are treated as `New York` - we can do this by creating a bigram model and modifying our corpus accordingly.

This code block creates a Phrases model from the input texts and then applies it to the texts to detect and join bigrams. The result is stored back in texts. The resulting bigrams are printed for the first item in texts.






In [ ]:
bigram = gensim.models.phrases.Phrases(texts)
texts = [bigram[line] for line in texts]
texts = [bigram[line] for line in texts]

print(texts[0])

['hundred', 'people', 'force', 'vacate', 'home', 'Southern', 'Highlands', 'New_South', 'Wales', 'strong', 'wind', 'push', 'huge', 'bushfire', 'town', 'Hill', 'new', 'blaze', 'near', 'Goulburn', 'south_west', 'Sydney', 'force', 'closure', 'Hume', 'Highway', '4:00pm', 'AEDT', 'marked', 'deterioration', 'weather', 'storm', 'cell', 'move', 'east', 'Blue_Mountains', 'force', 'authority', 'decision', 'evacuate', 'people', 'home', 'outlying', 'street', 'Hill', 'New_South', 'Wales', 'southern', 'highland', 'estimated', 'resident', 'leave', 'home', 'nearby', 'Mittagong', 'New_South', 'Wales', 'Rural_Fire', 'Service', 'weather_condition', 'cause', 'fire_burn', 'finger', 'formation', 'ease', 'fire', 'unit', 'Hill', 'optimistic', 'defend', 'property', 'blaze', 'burn', 'New', 'Year', 'Eve', 'New_South', 'Wales', 'fire', 'crew', 'call', 'new', 'fire', 'Gunning', 'south', 'Goulburn', 'detail', 'available', 'stage', 'fire', 'authority', 'close', 'Hume', 'Highway', 'direction', 'new', 'fire', 'Sydney',

This code block creates a gensim dictionary and corpus from the preprocessed texts. It prints the corpus for the second document in the texts list.






In [ ]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

print(corpus[1])

[(71, 1), (83, 1), (91, 1), (93, 1), (94, 1), (108, 1), (109, 1), (110, 1), (111, 4), (112, 1), (113, 1), (114, 1), (115, 1), (116, 2), (117, 1), (118, 1), (119, 3), (120, 1), (121, 1), (122, 1), (123, 2), (124, 3), (125, 1), (126, 2), (127, 2), (128, 1), (129, 1), (130, 1), (131, 1), (132, 1), (133, 1), (134, 1), (135, 1), (136, 1), (137, 1), (138, 3), (139, 1), (140, 1), (141, 1), (142, 2), (143, 1), (144, 1), (145, 1), (146, 1), (147, 1), (148, 1), (149, 1), (150, 3), (151, 3), (152, 1), (153, 1), (154, 2), (155, 1), (156, 1), (157, 2), (158, 1), (159, 1), (160, 1), (161, 1), (162, 1), (163, 1), (164, 1), (165, 1), (166, 1), (167, 1), (168, 1), (169, 1), (170, 2), (171, 1), (172, 1), (173, 1), (174, 1), (175, 1), (176, 1)]


> Now we are done with a very important part of text analysis - the data cleaning and setting up of corpus. It must kept in mind that we created the corpus the way we did because that's how gensim requires it - most algorithms still require one to clean the data set the way we did, by removing stop words and numbers, adding the lemmatized form of the word, and using bigrams.

---
# 📚 Topic Modeling

> Topic Modeling refers to the probabilistic modeling of text document as topics. Gensim remains the most popular library to perform such modelling, and we will be using it to perform our topic modelling.

## ✔️ LSI - Latent Semantic Indexing

> LSI stands for Latent Semantic Indexing - It is a popular information retreival method which works by decomposing the original matrix of words to maintain key topics. 

This code block is creating an LSI model using the input corpus and dictionary, then showing the top 5 topics with their associated words.






In [ ]:
lsi_model = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)
lsi_model.show_topics(num_topics=5)

[(0,
  '-0.231*"israeli" + -0.215*"Arafat" + -0.197*"palestinian" + -0.177*"force" + -0.159*"kill" + -0.159*"official" + -0.151*"attack" + -0.141*"people" + -0.118*"day" + -0.116*"Israel"'),
 (1,
  '0.306*"israeli" + 0.305*"Arafat" + 0.272*"palestinian" + -0.167*"Afghanistan" + 0.165*"Sharon" + -0.159*"Australia" + 0.154*"Israel" + 0.126*"Hamas" + 0.123*"West_Bank" + -0.118*"force"'),
 (2,
  '0.259*"Afghanistan" + 0.218*"force" + -0.190*"fire" + 0.182*"Al_Qaeda" + 0.173*"bin_Laden" + 0.150*"Pakistan" + -0.147*"Sydney" + 0.130*"fighter" + 0.130*"Tora_Bora" + 0.127*"Taliban"'),
 (3,
  '0.388*"fire" + 0.271*"area" + -0.208*"Australia" + 0.202*"Sydney" + 0.179*"firefighter" + 0.160*"north" + 0.149*"wind" + 0.135*"Wales" + 0.135*"New_South" + 0.127*"south"'),
 (4,
  '-0.273*"company" + -0.206*"Qantas" + -0.179*"union" + 0.165*"test" + -0.148*"worker" + 0.143*"match" + 0.141*"South_Africa" + 0.134*"win" + 0.131*"wicket" + 0.124*"day"')]

## ✔️ HDP - Hierarchical Drichlet Process

> HDP, the Hierarchical Drichlet Process is an unsupervised topic model which figures out the number of topics on it's own.

This code block uses the HDP (Hierarchical Dirichlet Process) model to train a topic model on the corpus and then shows the resulting topics.






In [ ]:
hdp_model = HdpModel(corpus=corpus, id2word=dictionary)
hdp_model.show_topics()

[(0,
  '0.007*israeli + 0.005*Arafat + 0.005*Sharon + 0.004*palestinian + 0.004*group + 0.003*Government + 0.003*attack + 0.003*kill + 0.003*choose + 0.003*West_Bank + 0.002*match + 0.002*Gaza_Strip + 0.002*official + 0.002*Hamas + 0.002*ask + 0.002*Palestinian_Authority + 0.002*terrorism + 0.002*play + 0.002*office + 0.002*militant'),
 (1,
  '0.004*airport + 0.003*Taliban + 0.002*Kandahar + 0.002*report + 0.002*night + 0.002*kill + 0.002*city + 0.002*opposition + 0.002*wound + 0.002*civilian + 0.002*Justice + 0.002*Agha + 0.002*Lali + 0.002*believe + 0.002*bombing + 0.002*royal_commission + 0.002*leave + 0.002*Gul + 0.002*early + 0.001*half'),
 (2,
  '0.004*storm + 0.003*tree + 0.002*Sydney + 0.002*work + 0.002*damage + 0.002*SES + 0.002*Energy + 0.002*area + 0.002*metre + 0.002*Hornsby + 0.002*home + 0.001*Australia + 0.001*bring + 0.001*yesterday + 0.001*hit + 0.001*bad + 0.001*emergency + 0.001*race + 0.001*night + 0.001*explain'),
 (3,
  '0.005*company + 0.004*cent + 0.003*job + 0

## ✔️ LDA - Latent Dirchlet Allocation

> LDA, or Latent Dirchlet Allocation is arguably the most famous topic modeling algorithm out there. Out here we create a simple topic model with 10 topics.

This code block performs topic modeling using LDA with 10 topics and displays the top terms for each topic.






In [ ]:
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
lda_model.show_topics()

[(0,
  '0.005*"year" + 0.004*"think" + 0.004*"attack" + 0.003*"know" + 0.003*"people" + 0.003*"United_States" + 0.003*"force" + 0.003*"Government" + 0.003*"official" + 0.003*"Arafat"'),
 (1,
  '0.003*"Israel" + 0.003*"year" + 0.003*"day" + 0.003*"people" + 0.003*"United_States" + 0.003*"Australia" + 0.003*"staff" + 0.002*"change" + 0.002*"virus" + 0.002*"claim"'),
 (2,
  '0.007*"day" + 0.006*"Australia" + 0.004*"force" + 0.004*"year" + 0.004*"man" + 0.003*"test" + 0.003*"area" + 0.003*"tell" + 0.003*"company" + 0.003*"Al_Qaeda"'),
 (3,
  '0.008*"israeli" + 0.005*"palestinian" + 0.005*"Arafat" + 0.005*"kill" + 0.004*"people" + 0.004*"Hamas" + 0.004*"attack" + 0.004*"metre" + 0.004*"win" + 0.004*"group"'),
 (4,
  '0.005*"report" + 0.005*"force" + 0.004*"people" + 0.004*"Australia" + 0.004*"Pakistan" + 0.003*"attack" + 0.003*"Afghanistan" + 0.003*"company" + 0.003*"India" + 0.003*"kill"'),
 (5,
  '0.006*"people" + 0.004*"new" + 0.004*"year" + 0.004*"Australia" + 0.004*"report" + 0.004*"Af

---
# 📊 pyLDAvis

Python library for interactive topic model visualization. This is a port of the fabulous R package by Carson Sievert and Kenny Shirley.

**pyLDAvis** is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

The visualization is intended to be used within an IPython notebook but can also be saved to a stand-alone HTML file for easy sharing.

In [ ]:
!pip install pyLDAvis==3.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 17.2 MB/s eta 0:00:00


This code block imports pyLDAvis.gensim and visualizes the LDA topic model using the pyLDAvis library in a Jupyter notebook.






In [ ]:
import pyLDAvis.gensim


pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.024092  0.059854       1        1  17.174159
5      0.024245  0.012757       2        1  16.683835
3     -0.022982  0.024070       3        1  12.770045
1     -0.030081 -0.021740       4        1  12.078344
4     -0.050370 -0.023501       5        1  10.322391
2     -0.010954  0.024614       6        1   8.109839
7     -0.004822 -0.016950       7        1   6.616360
0      0.046822 -0.054117       8        1   5.852322
9     -0.021260  0.001324       9        1   5.677346
8      0.045311 -0.006310      10        1   4.715359, topic_info=         Term       Freq      Total Category  logprob  loglift
2357  company  57.000000  57.000000  Default  30.0000  30.0000
838   israeli  77.000000  77.000000  Default  29.0000  29.0000
56       fire  64.000000  64.000000  Default  28.0000  28.0000
96      storm  21.000000  21.000000  Default  27.0000  27.0000
915      test  42.000000  42.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
217       day   2.859949  97.662138  Topic10  -6.1589  -0.4764
1017      man   2.673145  68.913155  Topic10  -6.2265  -0.1952
835   include   2.492954  46.900221  Topic10  -6.2962   0.1198
23     Sydney   2.407345  46.934478  Topic10  -6.3312   0.0841
331     claim   2.415989  49.868775  Topic10  -6.3276   0.0271

[811 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
637       1  0.210548           
637       2  0.147384           
637       3  0.084219           
637       4  0.147384           
637       5  0.105274           
...     ...       ...        ...
176       6  0.093437  yesterday
176       7  0.046719  yesterday
176       8  0.023359  yesterday
176       9  0.023359  yesterday
176      10  0.070078  yesterday

[2297 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 4, 2, 5, 3, 8, 1, 10, 9])

> This is a great way to get a view of what words end up appearing in our documents, and what kind of document topics might be present.